In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as MD

In [2]:
# Create Connection to postgresql
# psql -h localhost -p 25432 -U musicbrainz musicbrainz_db

conn = psycopg2.connect(
    host="localhost",
    database="musicbrainz_db",
    user="musicbrainz",
    port=5432)

cursor = conn.cursor()

cursor.execute("select * from recording limit 1;")
cursor.fetchone()[0]
print("connection successful")

connection successful


In [3]:
%%time
# Loading recording table SQL tables

MB_recording = pd.read_sql('SELECT gid FROM recording', con = conn, columns = ['rec-gid'])
# MB_recording.set_index('gid', inplace = True)
MB_recording.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 6.91 s, sys: 2.68 s, total: 9.59 s
Wall time: 18.5 s


,gid
0,0f42ab32-22cd-4dcf-927b-a8d9a183d68b
1,4dce8f93-45ee-4573-8558-8cd321256233
2,48fabe3f-0fbd-4145-a917-83d164d6386f
3,b30b9943-9100-4d84-9ad2-69859ea88fbb
4,b55f1db3-c6d2-4645-b908-03e1017a99c2


In [4]:
%%time
# Loading redirect table SQL table

# MB_redirects = pd.read_sql('select r.gid, rgr.gid from recording r join recording_gid_redirect rgr on rgr.new_id = r.id', con = conn)
# MB_redirects.columns = ['old', 'new']

MB_redirects = pd.read_sql('select r.gid AS new, rgr.gid AS old from recording r join recording_gid_redirect rgr on rgr.new_id = r.id', con=conn)
MB_redirects.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 1.38 s, sys: 166 ms, total: 1.55 s
Wall time: 7.12 s


,new,old
0,d04bb774-5104-4cc2-9c70-1e76bd9917e1,841bdd72-f415-4731-8e37-71b53db18d93
1,ddda2877-0fbd-495e-a19b-6e9f4e97d711,8edeb408-437d-4e97-93f8-2fb982927fb0
2,6db0efa0-46e0-4f27-94ce-6803513787be,de4ed46b-a78b-4bd4-94f5-82f5829931e0
3,8dda3e70-02f3-4032-871c-b16ba184a9f6,89d8fd1e-7372-42cf-8ab0-6d4d46576a86
4,af42602a-a32f-463a-8283-ea4c8bace06c,0d806561-1121-43c9-a440-5a5c7d3be82d


In [5]:
%%time
# Loading track table

MB_track = pd.read_sql('SELECT gid FROM track', con = conn)
MB_track.columns = ['track-gid']
MB_track.set_index('track-gid', inplace=True)
MB_track.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 9.62 s, sys: 3.44 s, total: 13.1 s
Wall time: 25.2 s


""
track-gid
9b02977e-a03b-4a6b-a9a9-06e722bdcd7a
43da7544-6283-3159-84f9-537fe823a1a7
0b6b6283-a5a8-4560-9fa8-f68a430d86ea
fa124f9a-d8ea-36a3-bed3-c817fdbe13e2
e56c6d3c-09cf-33a0-81c5-ceade77c35dc


In [6]:

%%time
# Loading canonical recording mbid table

MB_canonical = pd.read_sql('SELECT recording_mbid, canonical_recording_mbid FROM mapping.canonical_recording_redirect', con = conn)
MB_canonical.columns = ['old', 'new']
# MB_canonical.set_index('recording-gid', inplace=True)
MB_canonical.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 2.29 s, sys: 769 ms, total: 3.06 s
Wall time: 6.12 s


,old,new
0,6ac02452-ee12-4f86-b389-bd20ba2fefcf,3e8eebfd-7613-4b3d-acbe-41709be76618
1,b4c26989-1b9e-4d50-8cde-56d6472e4bc3,3e8eebfd-7613-4b3d-acbe-41709be76618
2,601e1cf3-ad6c-4e38-9128-ba4d0d4b010f,b1050d12-b8af-409c-9cff-22759d93e240
3,35c4d840-e51f-4c07-9418-af9335b29642,f4680747-bf28-417a-ab33-af00577d8ac2
4,9ba7a9b9-a21c-4b12-8771-4c108b08b3e2,13b3875a-c89a-4be5-a6e4-0ca9164bc41d


In [7]:
%%time
# Loading artist-mbid table for artist-conflation.

MB_artist = pd.read_sql('SELECT gid FROM artist', con = conn)
MB_artist.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 550 ms, sys: 100 ms, total: 650 ms
Wall time: 1.34 s


,gid
0,fadeb38c-833f-40bc-9d8c-a6383b38b1be
1,49add228-eac5-4de8-836c-d75cde7369c3
2,c112a400-af49-4665-8bba-741531d962a1
3,ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b
4,7b4a548e-a01a-49b7-82e7-b49efeb9732c


# Loading Data

In [8]:
# Reads a list of file paths and reads + compiles data into a single pd.DataFrame
def read_files(file_path_repo):
    
    # init new empty main dataframe
    df = pd.DataFrame(columns = ['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
    
    # Open a file with MLHD file paths to process
    with open(file_path_repo, 'r') as f:
        file_paths = f.readlines()
        file_paths= [item.strip() for item in file_paths]
    
    # Read files and compile into single df
    for pth in file_paths:
        temp = pd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
        temp = temp[-temp['recording-MBID'].isna()]

        df = pd.concat([df, temp])
    
    return df

df = read_files('random_file_paths.txt')
print(df.shape)
df.head()

(3661057, 4)


,timestamp,artist-MBID,release-MBID,recording-MBID
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,426c5c82-4472-4f7d-b1d9-9f928d338340,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,6079df6b-2c00-4fd6-b015-0e303eedf4fd,14e9eb4e-155d-46ff-9a83-a8d5e1936c81
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89
3,1108428897,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89
4,1108429140,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89


In [9]:
null_count_artist = df['artist-MBID'].isnull().value_counts()
null_count_rec = df['recording-MBID'].isnull().value_counts()
null_count_rel = df['release-MBID'].isnull().value_counts()

def get_null_stats(val_count, attr_name):
    print("Number of NOT-null rows in {} = {}".format(attr_name, (val_count[0]/val_count.sum())*100))

get_null_stats(null_count_artist, 'artist-MBID')
get_null_stats(null_count_rec, 'recording-MBID')
get_null_stats(null_count_rel, 'release-MBID')

Number of NOT-null rows in artist-MBID = 98.87100911021052
Number of NOT-null rows in recording-MBID = 100.0
Number of NOT-null rows in release-MBID = 75.44897552810568


## Architechture:

1. Take a chunk of MBIDs (Test optimal chunk sizes too. Current optimal > 253k rows)
2. "Squish" series
    - i.e. Only take unique values from the series.
3. Pass squished series thorugh the following:
    - Get mbids, and check if they exist in the recording table.
    - Get mbids that don't exist in redirect, and pass it through MB_track
    - Get mbids that don't exist in recording, and pass it through MB_redirect
    - get mbids that don't exist in track.... There's no MBIDs that belong to track.
4. "Unsquish" the series.
    - i.e. Take processed output for squished values, and apply them to unsquished values.
    - This process ensures processing only on unique values.
    The output for this processing is then applied to duplicate values as well.

In [10]:
# '''Squish function: 
# 1. Takes in input series with index number and recording-MBID.
# 2. Makes a mapping table with recording-MBIDs as the index, 
# and a series of row-indices with that MBID as the values.'''

# # def squish(input_series):

# '''
# 1. take inp_series
# 2. Generate empty mapping_df where:
#     - index = MBID
#     - value = series of indices from inp_series
# 2. start traversing
# 3. if new ID: 
#     - Add ID to mapping_df.index
#     - Set the value in mapping_df as a list of newly updated indices for inp_series
# 4. if not new ID:
#     - Add ID to list of indices for inp_series.
#     - Update this ID in mapping_df.index
# '''

# # inp = df['recording-MBID'].reset_index(drop=True)
# # inp_unique = inp.unique()

# # mapping_df = pd.DataFrame(index=inp_unique)
# # mapping_df

# # Tackle this later. Focus on basic unoptimized code first!

In [11]:
# A generic function for queries a series into another series.
    # Returns a series of boolean values corresponding to series_of_mbids 
    # (bool specifies if value exists in recording table or not.)

def query_in(series_to_query, series_to_query_in):
    # Queries all mbids in the recording table. Returns mbids that are present in recording table.
    mbids_in_series = series_to_query_in[series_to_query_in.isin(series_to_query)]
    
    # Makes a boolean map for all mbids in series_of_mbids
    bool_map = series_to_query.isin(mbids_in_series)
    
    return bool_map

# A generic function to get corresponding values to a query.
def get_query(series_to_query, df_to_query_in):
    mbids_in_series = df_to_query_in[df_to_query_in.index.isin(series_to_query)]
    
    return mbids_in_series

## Testing Track MBIDs

In [12]:
%%time

# Just to prove that absolutely none of the of the 253k unique mbids are track-mbids disguised as rec-mbid.

# in_for_tack = in_for_rec[-out_for_rec]
in_for_track = pd.Series(df['recording-MBID'].unique())
out_for_track = query_in(in_for_track, MB_track)

track_mbid_in_r_mbid = out_for_track.value_counts()
track_mbid_in_r_mbid

CPU times: user 16.5 s, sys: 36.1 ms, total: 16.6 s
Wall time: 16.6 s


False    381145
dtype: int64

## Testing for recording-MBID

In [13]:
%%time
# Testing query_in() with recording MBID

in_for_rec = pd.Series(df['recording-MBID'].unique())
out_for_rec = query_in(in_for_rec, MB_recording.gid)

r_mbid_in_r_mbid = out_for_rec.value_counts()
r_mbid_in_r_mbid

CPU times: user 9.26 s, sys: 0 ns, total: 9.26 s
Wall time: 9.27 s


True     295224
False     85921
dtype: int64

## Testing for redirects

In [14]:
%%time

# Testing if MBIDs that DONT exist in MB recording table have any redirects
in_for_redir = in_for_rec[-out_for_rec]

out_for_redir = query_in(in_for_redir, MB_redirects.old)

r_mbid_in_redir = out_for_redir.value_counts()
r_mbid_in_redir

CPU times: user 804 ms, sys: 0 ns, total: 804 ms
Wall time: 800 ms


True     84773
False     1148
dtype: int64

In [15]:
%%time

# Testing if MBIDs that exist in MB recording table have any redirects
in_for_redir = in_for_rec[out_for_rec]

out_for_redir = query_in(in_for_redir, MB_redirects.old)

r_mbid_in_redir = out_for_redir.value_counts()
r_mbid_in_redir

CPU times: user 705 ms, sys: 0 ns, total: 705 ms
Wall time: 702 ms


False    295224
dtype: int64

In [16]:
# # Fetching redirected MBIDs.

# # in_for_redir = pd.Series(df['recording-MBID'].unique())
# in_for_redir = in_for_rec[out_for_rec]
# out_for_redir = MB_redirects.old.isin(in_for_redir)

# out_for_redir.value_counts()

## Checking if artist-MBIDs belong to artist ID.

In [17]:
# Checking if artist-MBIDs belong to artist ID.

input_artists = pd.Series(df['artist-MBID'].unique())

output = query_in(input_artists, MB_artist.gid)
output.value_counts()

True     30839
False     2597
dtype: int64

# Analysis Report

- **Total Input**:
    - Total Sample Size: 3,661,057 rows
    - Unique Rows: 381145
    - Number of NOT-null rows in artist-MBID = 98.87100911021052
    - Number of NOT-null rows in recording-MBID = 100.0
    - Number of NOT-null rows in release-MBID = 75.44897552810568


- **Number of rec-MBIDs in track-MBID**:
    - Total Input: 381145
    - False: 100%
    (no rec-MBID was found in the track-MBID)


- **Number of rec-MBIDs in recording-MBID**:
    - Total Input: 381145
    - (True, False) = (295224, 85921)
    - i.e. 77.457% rec-MBIDs belong to the recording-MBID table.


- **Number of UNKNOWN rec-MBIDs in redirected-MBID**:
    - Note: "UNKNOWN rec-MBIDs" = rec-MBIDs from the MLHD that were NOT found in the MB recording table.
    - Total Input: 85921 rows
    - (True, False): (84773, 1148)
    - i.e. 98.66% of UNKNOWN rec-MBIDs have corresponding redirects.


- **Number of KNOWN rec-MBIDs in redirected-MBID**:
    - Note: "KNOWN rec-MBIDs" = rec-MBIDs from the MLHD that were found in the MB recording table.
    - Total Input = 295224 rows
    - False = 295224 rows
    - i.e. 100% KNOWN rec-MBIDs don't have any redirects.


- **Number of rec-MBIDs in canonical-MBID table**:
    - _pending_


- **Number of artist-MBIDs in MB artist table**:
    - Total Input: 33436
    - True, False = (30839, 2597)
    - i.e. 92.23% artist-MBIDs exist in the MB artist table.